# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import re
from datetime import datetime, timedelta
import math


# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## Проверка на уникальность
Проверим полностью дублирующиеся строки



In [ ]:
data.duplicated(subset=None, keep='first').value_counts()

Полностью дублирующихся строк нет.

Проверим отдельные поля датасета на наличие повторений и дубликатов

### 'Restaurant_id'


In [ ]:
len(data['Restaurant_id'].value_counts().loc[lambda x : x > 1].index)

Как видно есть повторяющиеся значения. Выясним, что они означают.

In [ ]:
data['Restaurant_id'].value_counts().loc[lambda x : x > 1].index[:10]

Строки с одинаковым 'Restaurant_id' имеют различные, не повторяющиеся поля как городов, типов кухонь, так и идентификаторов ID_TA. Можно предположить, что это не ошибка данных,  скорее всего поле означает  не конкретный ресторан, а сеть  ресторанов или одного владельца.


### 'City'

In [ ]:
data['City'].value_counts().sort_index(ascending=True)

In [ ]:
cities_list = list(data['City'].value_counts().index)

C названиями городов все в порядке

## 'Cuisine Style'

У данного признака присутствую пропуске в наборе данных.

Посмотрим более детально какие кухни преобладают в общей выборке и отдельно по городам.

In [ ]:
data['Cuisine Style'].value_counts(dropna=False).head(20)

In [ ]:
data[['City', 'Cuisine Style']].groupby('City').describe()

In [ ]:
data[['City', 'Cuisine Style']].groupby('City')['Cuisine Style'].value_counts()['London'].head(10)

In [ ]:
data[['City', 'Cuisine Style']].groupby('City')['Cuisine Style'].value_counts()['London'].head(1).index[0]

Посчитаем количество встречающихся стилей кухонь в предлагаемых наборах стилей.

In [ ]:
cuisine_dict = {}
styles_count = []
for style_set in data[data['Cuisine Style'].notna()]['Cuisine Style']:
    style_set_list = style_set.replace('"', '').replace('[', '').replace(']', '').replace("'", "").\
                    replace(', ', ',').split(',') 
    styles_count.append(len(style_set_list))
    for cuisine_style in style_set_list:
        if cuisine_style in cuisine_dict:
            cuisine_dict[cuisine_style] += 1
        else:
            cuisine_dict[cuisine_style] = 1

sorted_cuisine_dict = {}
sorted_keys = sorted(cuisine_dict, key=cuisine_dict.get)  # [1, 3, 2]

for w in sorted_keys:
    sorted_cuisine_dict[w] = cuisine_dict[w]

sorted_cuisine_dict

## 'Price Range'

In [ ]:
data['Price Range'].value_counts(dropna=False)

In [ ]:
data[['City', 'Price Range']].groupby('City').describe()

Как видно ценовой диапазон, что в общей выборке, что по городам в основном средний '\$\$ \- \$\\$\\$'.

## 'Number of Revews'

In [ ]:
data['Number of Reviews'].value_counts(dropna=False)

In [ ]:
data['Number of Reviews'].describe()

In [ ]:
data[['City', 'Number of Reviews']].groupby('City').describe()

## 'Reviews'

In [ ]:
len(data['Reviews'].value_counts().loc[lambda x : x > 1])

In [ ]:
data['Reviews'].value_counts().loc[lambda x : x > 1]

В датасете встречаются дублированные отзывы. Количество повторов не более двух. Также 8112 записей с пустыми отзывами

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')
data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('uint8')
data['Price_Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')

In [ ]:
# Создадим класс для обработки пропусков объектами которого можно формировать 
# различные способы заполнения пропусков

class FillNaByFeature():
    def __init__(self, feature, value, data):
        self.df = data[[feature, value]].groupby(feature).describe()
        self.feature = feature
        self.value = value
    
    def fill_na(self, row, statistic=0):
        #if np.isnan(row[self.value]):
        if pd.isnull(row[self.value]):
            if statistic == 'mean':
                return self.df[self.value, 'mean'][row[self.feature]]
            if statistic == 'median':
                return self.df[self.value, '50%'][row[self.feature]]
            if statistic == 'top':
                return self.df[self.value, 'top'][row[self.feature]]
            if statistic == 0:
                return 0
        else:
            return row[self.value]

In [ ]:
# Создаем объект для заполнения пропусков признака 'Number of Reviews' по городу
filling_number_of_review = FillNaByFeature('City', 'Number of Reviews', data )

In [ ]:
# Создадим объект для заполнения пропусков признака 'Price Range' по городу
filling_price_range = FillNaByFeature('City', 'Price Range', data)

In [ ]:
# Заполним пропуски 'Number of Reviews' медианой по городу
data['Number of Reviews'] = data.apply(filling_number_of_review.fill_na, args=('median',), axis=1)

In [ ]:
# Заполним пропуски 'Price Range' часто встречающимся значением по городу
data['Price Range'] = data.apply(filling_price_range.fill_na, args=('top',), axis=1)

In [ ]:
# Создадим объект для заполнения пропусков признака 'Cuisine Style' по городу
filling_cuisine_style = FillNaByFeature('City', 'Cuisine Style', data)

In [ ]:
# Заполним пропуски 'Cuisine Style' часто встречающимся значением по городу
data['Cuisine Style'] = data.apply(filling_cuisine_style.fill_na, args=('top',), axis=1)

In [ ]:
# Заполнил вначале пропуски в поле 'Reviews' пустым значением как и у других записей с пустыми обзорами. Добавим отдельное поле 'Reviews_isNaN' для пустых обзоров.
# Затем распарсим значения на отдельные отзывы и их даты.
data['Reviews'] = data['Reviews'].fillna('[[], []]')

In [ ]:
data['Reviews_isNaN'] = data['Reviews'].apply(lambda x: 1 if x == '[[], []]' else 0)

In [ ]:
data.info()

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

In [ ]:
# Обработка 'Price Range'

data['Price Range'] = data['Price Range'].apply(lambda x: 3 if x == '$$$$' else x)
data['Price Range'] = data['Price Range'].apply(lambda x: 2 if x == '$$ - $$$' else x)
data['Price Range'] = data['Price Range'].apply(lambda x: 1 if x == '$' else x)

In [ ]:
# Обработка признака 'Cuisine Style'

data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: x.replace('"', '').replace('[', '').replace(']', '').replace("'", "").replace(', ', ',').split(','))
df_cuisine_style = data['Cuisine Style'].str.join('@').str.get_dummies('@')
data = pd.concat([data, df_cuisine_style], axis=1)

In [ ]:
# Добавим дополнительный признак - количество стилей кухонь в ресторане

data['cuisine_style_count'] = data['Cuisine Style'].apply(lambda x: len(x))

In [ ]:
# Функция для парсинга поля 'Reviews' 
# Входные данные: строка, индекс (0 - 3)
# Выходные данные: Строки обзора, либо даты

def review_parsing(input_str, index=""):
    input_str = input_str.replace("[", "").replace("]", "")
    begin_item = True
    limiter = ""
    out_list = []
    out_str =""
    limiter_count = 0
    for symbol in enumerate(input_str):
        if begin_item and symbol[1] != ' ' and symbol[1] != ',' and limiter_count == 0:
            limiter = symbol[1]
            begin_item = False
            limiter_count += 1
        elif symbol[1] == limiter:
            if symbol[0] < len(input_str) - 1 and symbol[0] > 0:
                if input_str[symbol[0] + 1] == ',' and (input_str[symbol[0] - 1] != '\\' or \
                                                        input_str[symbol[0] - 1] == '\\' and input_str[symbol[0] - 2] == '\\'):
                    limiter_count += 1
            else:
                limiter_count += 1
            if limiter_count == 2:
                out_list.append(out_str)
                out_str = ""
                begin_item = True
                limiter_count = 0
        elif limiter_count == 1:
            out_str += symbol[1]
    if len(out_list) == 0:
        out_list =  [np.NaN, np.NaN, np.NaN, np.NaN]
    elif len(out_list) == 2:
        out_list.insert(1, np.NaN)
        out_list.append(np.NaN)
    if index == '':
        return out_list
    else:
        return out_list[index]

In [ ]:
# Из данных 'Reviews' извлечем последний отзыв
data['last_review'] = data['Reviews'].apply(review_parsing, index=0)

In [ ]:
# Из данных 'Reviews' извлечем дату последнего отзыва
data['last_review_date'] =  pd.to_datetime(data['Reviews'].apply(review_parsing, index=2))

In [ ]:
# Из данных 'Reviews' извлечем предпоследний отзыв
data['prev_review'] = data['Reviews'].apply(review_parsing, index=1)

In [ ]:
# Из данных 'Reviews' извлечем дату предпоследнего отзыва
data['prev_review_date'] = pd.to_datetime(data['Reviews'].apply(review_parsing, index=3))

In [ ]:
# Создадим новый признак - количество дней прошедших между последним и предпоследним отзывом
data['review_days_elasped'] = (data.last_review_date - data.prev_review_date).apply(lambda x: abs(x.days))

In [ ]:
data[['Reviews', 'last_review', 'last_review_date', 'prev_review', 'prev_review_date', 'review_days_elasped' ]]

In [ ]:
# Создадим новые признаки, кол-во символов в отзыве, пропущенные отзывы заполним нулями
data['last_review_len'] = data['last_review'].apply(lambda x: len(x) if pd.notnull(x) else 0)
data['prev_review_len'] = data['prev_review'].apply(lambda x: len(x) if pd.notnull(x) else 0)

In [ ]:
data['review_days_elasped'] = data['review_days_elasped'].fillna(9999)

In [ ]:
data.info(verbose=True, null_counts=True)

In [ ]:
#новые полиномиальные признаки
data['number_reviews_ranking'] = data['Number of Reviews'] * data['Ranking']
data['numberofr_rank'] = data['Number of Reviews'] / data['Ranking']

In [ ]:
#функции определения положительных и отрицательных слов в отзывах
def find_good_words(s):
    """
    Выявление количества положительных слов в отзывах
    """
    good_words = ['good', 'yummy', 'fine', 'great', 'tasty',
                  'satisfaction', 'amazing', 'nice', 'best',
                  'friendly', 'pleasant', 'exellent', 'loved',
                  'love', 'lovely', 'welcoming', 'wonderful',
                  'perfect', 'delicious', 'favourite', 'sweet',
                  'yum', 'adequate', 'happy', 'beautiful', 'liked', 
                  'like', 'finest', 'greatest', 'excellent', 'paramount', 
                  'preeminent', 'superlative', 'top', 'unsurpassed', 
                  'amazing', 'astonishing', 'awe-inspiring', 'awesome', 
                  'awful', 'awing']
    count = 0
    for item in good_words:
        if item in s:
            count += 1
    return(count)

def find_bad_words(s):
    """
    Выявление количества отрицательных слов в отзывах
    """
    bad_words = ['bad', "poor", "ill", "low", "inferior", "wretched",
                 'badly', 'awry', "unpalatable", "unappetizing", "unsavory",
                 "istasteful", "unpleasant", "nasty", "dirty", "rough", "roughly", "tough",
                 "ugly", "graceless", "rudeness", 'boorishness', 'loutishness', 
                 'disagreeable', 'nasty', 'soiled', 'sordid', 'unclean', 'unwashed',
                 'indecent', 'lewd', 'obscene', 'raunchy', 'salacious',
                 'cheating', 'foul', 'unsporting', 'unsportsmanlike']
    count = 0
    for item in bad_words:
        if item in s:
            count += 1
    return(count)

data['Good_rev'] = data.Reviews.str.lower().apply(find_good_words)
data['Bad_rev'] = data.Reviews.str.lower().apply(find_bad_words)
data['Rev_delta'] = data['Good_rev'] - data['Bad_rev']

In [ ]:
# убираем не нужные для модели признаки
data.drop(['Restaurant_id', 'Cuisine Style', 'Reviews', 'ID_TA','URL_TA', \
                 'last_review', 'last_review_date', 'prev_review', 'prev_review_date'], axis = 1, inplace=True)

In [ ]:
data.info(verbose=True, null_counts=True)

In [ ]:
# Создание dummi-переменных по колонке 'City'
data = pd.get_dummies(data, columns=['City'], dummy_na=True)

### Графики и диаграммы

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=150)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

### Распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

# Data Preprocessing

In [ ]:
df_preproc = data

In [ ]:
df_preproc.info()

In [ ]:
# Выделение тестовой части
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)